In [1]:
# korekta błędu w Keras po zmianie w bibliotece numpy
import numpy as np
np_load_old = np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

# wyłączenie ostrzeżeń
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Deep Dream


----

[...]

## Implementacja algorytmu DeepDream w pakiecie Keras

Zaczniemy od konwolucyjnej sieci neuronowej wytrenowanej na zbiorze obrazów ImageNet. Pakiet Keras zawiera wiele takich sieci. Są to między innymi: VGG16, VGG19, Xception i ResNet50. Algorytm DeepDream może zostać zaimplementowany przy użyciu każdej z tych sieci, ale wybór sieci będzie miał oczywiście wpływ na generowane wizualizacje. Wynika to z tego, że różne architektury sieci konwolucyjnych uczą się różnych cech. W oryginalnym algorytmie DeepDream zastosowano model Inception. Wykorzystanie tego modelu pozwala na wygenerowanie ładnie wyglądających grafik, a więc skorzystamy z modelu Inception V3 dołączonego do pakietu Keras.


In [3]:
from keras.applications import inception_v3
from keras import backend as K

# Nie będziemy trenować modelu. Polecenie to wyłącza wszystkie operacje używane tylko podczas trenowania.
K.set_learning_phase(0)

# Sieć Inception V3 jest budowana bez swojej konwolucyjnej bazy. 
# Model zostanie załadowany z wagami wytrenowanymi na zbiorze ImageNet.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)


Następnie musimy zająć się obliczaniem straty — wartości, którą będziemy starali się maksymalizować w procesie wzrostu gradientu. W rozdziale 5. podczas filtrowania wizualizacji staraliśmy się maksymalizować wartość określonego filtra wybranej warstwy sieci. Tym razem będziemy jednocześnie maksymalizować aktywacje wszystkich filtrów wielu warstw, a konkretnie rzecz biorąc, będziemy maksymalizować sumę normy L2 aktywacji zbioru warstw wysokiego poziomu. Wybór warstw (a także dokładanie się poszczególnych warstw do finalnej wartości straty) ma największy wpływ na generowane wizualizacje. W związku z tym chcemy, aby parametry te można było z łatwością modyfikować. Niższe warstwy odpowiadają za wzorce geometryczne, a wyższe warstwy odpowiadają za elementy obrazu pozwalające na rozpoznawanie klas zbioru ImageNet (np. ptaków lub psów). Zaczniemy od niezbyt optymalnej konfiguracji czterech warstw, ale z pewnością warto wypróbować później działanie wielu innych konfiguracji.

In [4]:
# Słownik przypisujący nazwy warstw do współczynników wpływu aktywacji warstw 
# na wartość straty, którą chcemy maksymalizować.
# Zauważ, że nazwy warstw są wprowadzone na stałe w wbudowanej aplikacji Inception V3. 
# Listę nazw wszystkich warstw modelu
# można wyświetlić za pomocą polecenia model.summary().
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

Teraz czas zdefiniować tensor zawierający wartość straty: ważoną sumę normy L2 aktywacji warstw z listingu 8.9.

In [5]:
# Tworzy słownik przypisujący nazwy warstw do instancji warstw.
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Strata będzie definiowana przez dodanie wartości charakteryzujących wpływ poszczególnych warstw na stratę.
loss = K.variable(0.)
for layer_name in layer_contributions:
    # Przechwytuje wyjście warstwy.
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    # Dodaje normę L2 cech warstwy do straty. Wpływ granicznych artefaktów jest pomijany poprzez określanie straty na podstawie pikseli nieznajdujących się na granicy.
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

Teraz możemy uruchomić proces wzrostu gradientu:

In [6]:
# W tym tensorze znajduje się wygenerowany obraz (wizja).
dream = model.input

# Oblicza gradienty wizji na podstawie wartości straty.
grads = K.gradients(loss, dream)[0]

# Normalizuje gradienty (to ważny zabieg).
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Konfiguruje funkcję Keras służącą do uzyskiwania wartości straty i gradientów na podstawie obrazu wejściowego.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Wartość straty', i, ':', loss_value)
        x += step * grad_values
    return x


Na koniec możemy zająć się właściwym algorytmem DeepDream. Na początku definiowana jest lista skal (określanych również mianem oktaw), które są używane podczas przetwarzania obrazów. Każda kolejna skala jest większa od poprzedniej o współczynnik równy 1,4 (jest o 40% większa) — zaczynamy od przetwarzania małego obrazu, a następnie zwiększamy jego skalę (patrz rysunek 8.4):

![proces deep dream](img\8_2.png)


Po każdej kolejnej operacji skalowania (od najmniejszej do największej) uruchamiany jest algorytm wzrostu gradientu w celu maksymalizacji zdefiniowanej wcześniej straty przy danej skali. Po każdym zakończeniu pracy tego algorytmu skala obrazu jest zwiększana o 40%.

W celu uniknięcia utraty dużej ilości szczegółów obrazu po każdej operacji skalowania (w wyniku tych operacji otrzymywany jest coraz bardziej rozmyty i rozpikselowany obraz) możemy wykonać prosty zabieg polegający na ponownym dodaniu utraconych szczegółów do obrazu. Jest to możliwe do wykonania, ponieważ wiemy, jak powinien wyglądać oryginalny obraz w większej rozdzielczości. Dysponując obrazem S o małym rozmiarze i obrazem L o większym rozmiarze, możemy przekształcić obraz L do rozmiaru obrazu S i określić różnice między tymi obrazami — różnica ta będzie określać utracone szczegóły.

W kodzie algorytmu zwiększania gradientu zastosowano poniższe funkcje pomocnicze Numpy. Do prawidłowej pracy wymagają one zainstalowania biblioteki SciPy.

In [7]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Funkcja narzędziowa konwertująca tensor do postaci właściwego obrazu,
    # zmiany jego rozdzielczości i zapisywania go w formie tensora, który może zostać przetworzony przez sieć Inception V3.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Funkcja narzędziowa konwertująca tensor do formy właściwego obrazu.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [8]:
import PIL
import numpy as np
import scipy
import scipy.misc
import imageio

# Modyfikacja tych parametrów pozwala na uzyskanie innych efektów wizualnych.

step = 0.01  # Rozmiar kroku algorytmu wzrostu gradientu.
num_octave = 3  # Liczba operacji skalowania, przy których należy uruchomić algorytm wzrostu gradientu.
octave_scale = 1.4  # Różnica między rozmiarami kolejnych wersji obrazu.
iterations = 20  # Liczba kroków wzrostu wykonywanych przy każdej operacji skalowania.

# Jeżeli strata przekroczy wartość równą 10,
# to proces wzrostu gradientu zostanie przerwany w celu zapobiegnięcia powstawania brzydkich artefaktów.
max_loss = 10.

# Tu należy umieścić ścieżkę obrazu, który chcemy przetwarzać.
base_image_path = 'creative_commons_elephant.jpg'

# Ładowanie obrazu do tablicy Numpy.
img = preprocess_image(base_image_path)

# Przygotowywanie listy krotek kształtów definiujących skalowania, 
# przy których uruchomiony zostanie algorytm wzrostu gradientu.
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Odwracanie listy kształtów tak, aby znalazły się one w kolejności rosnącej.
successive_shapes = successive_shapes[::-1]

# Zmiana rozmiaru tablicy Numpy obrazu w celu zmniejszenia jego skali.
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Zmiana kształtu obrazu', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

#save_img(img, fname='final_dream.png')

Zmiana kształtu obrazu (306, 458)
...Wartość straty 0 : 1.8070966
...Wartość straty 1 : 2.3078444
...Wartość straty 2 : 3.0236685
...Wartość straty 3 : 3.7816033
...Wartość straty 4 : 4.544466
...Wartość straty 5 : 5.263198
...Wartość straty 6 : 5.947104
...Wartość straty 7 : 6.566756
...Wartość straty 8 : 7.187997
...Wartość straty 9 : 7.7423105
...Wartość straty 10 : 8.339379
...Wartość straty 11 : 8.888827
...Wartość straty 12 : 9.429738
...Wartość straty 13 : 9.950428
Zmiana kształtu obrazu (428, 642)
...Wartość straty 0 : 3.0268502
...Wartość straty 1 : 4.399698
...Wartość straty 2 : 5.51843
...Wartość straty 3 : 6.491634
...Wartość straty 4 : 7.4079676
...Wartość straty 5 : 8.236033
...Wartość straty 6 : 8.985122
...Wartość straty 7 : 9.711582
Zmiana kształtu obrazu (600, 899)
...Wartość straty 0 : 2.974997
...Wartość straty 1 : 4.3280463
...Wartość straty 2 : 5.48747
...Wartość straty 3 : 6.4943824
...Wartość straty 4 : 7.4341903
...Wartość straty 5 : 8.304633
...Wartość straty 

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(deprocess_image(np.copy(img)))
plt.show()